# <ins>Analyzing Stellar Data from MESA</ins>
## An Interactive Exercise

In this exercise, you will carry out some basic analyses of stellar data from MESA. 


## Installing PyMesaReader

You will need to have PyMesaReader installed. To intall PyMesaReader, follow the first cell below. _The installation needs to be executed only once. Once PyMesaReader is installed, there is no need to install it again._

In [ ]:
import os

# Store the current working directory
cwd = os.getcwd()

# Check the py_mesa_reader code out from github to the user home directory
!git clone https://github.com/wmwolf/py_mesa_reader.git ~/py_mesa_reader

# Change into the install directory and use PIP to install the Python package
!cd ~/py_mesa_reader; pip3 install --user .

# Change back to the original working directory
os.chdir (cwd)

## Plotting MESA history data

For our first example, we will make use of the history file to plot the time evolution of the MESA stellar model on a theorist's HR diagram of log luminosity versus log T. 

Many other analyses are possible with the history data. The command 

<code>
h.bulk_names
</code>

below returns all of the fields stored. The <code>'LOGS/history.data'</code> file contains within it many global quantities, representing one value per timestep of the simulation. Small modifications can easily be made, for example, to plot the log luminosity versus time, or the log mass versus log radius of the star.

In [ ]:
import matplotlib.pyplot as plt
import mesa_reader as mr

# Set the MESA directory which needs to be analyzed
mesa_run_directory = "/home/local/UMDAR/rfisher1/mesa/tutorial"
os.chdir (mesa_run_directory)

# Display graphics inline in this notebook
%pylab inline
pylab.rcParams['figure.figsize'] = (20, 12)

# Read in the history file
h = mr.MesaData ('LOGS/history.data')

# To display all available fields
print ("This history.data file contains these fields: ", h.bulk_names)

ages = h.star_age
log_Teff = h.log_Teff
log_L = h.log_L

# Generate a "theorist's" HR diagram of log luminosity versus log T

font_size = 24

plt.plot (log_Teff, log_L, color = 'black', linewidth = 2.5)
plt.xlabel ("Log Effective Temperature (K)", fontsize = font_size)
plt.ylabel ("Log Luminosity ($L_{\odot}$)", fontsize = font_size)
plt.tick_params(axis='x', labelsize = font_size)
plt.tick_params(axis='y', labelsize = font_size)
# reverse temperature axis for HR diagram
ax = plt.gca()
ax.set_xlim(ax.get_xlim()[::-1])

# Output the HR diagram to a file in PDF format
plt.savefig ("mesa_hr_diagram.pdf", format = "pdf")